In [2]:
import numpy as np
import pandas as pd
%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_probability as tfp
# !pip install sklearn
# !pip install tensorflow
# !pip install edward2
from tensorflow_probability import edward2 as ed
from keras.utils import to_categorical
import tensorflow.keras as keras
import seaborn as sns



Using TensorFlow backend.


**Data for Sanity Check**


In [0]:
x = np.random.randn(140, 2)
Y = np.tanh(x[:, 0]+ x[:, 1])
Y = 1. / (1. + np.exp(-(Y+Y)))
Y = Y > 0.5

In [0]:
X_1 = x[Y]
X_0 = x[np.logical_not(Y)]

sns.scatterplot(X_1[:, 0], X_1[:, 1])
sns.scatterplot(X_0[:, 0], X_0[:, 1])

**Two Circles Dataset**

In [0]:
from sklearn.datasets import make_circles

t = make_circles(n_samples=1000, noise=0.04)
labels = t[1]
data = t[0]

cls_1 = data[labels == 1]
cls_0 = data[np.logical_not(labels)]

sns.scatterplot(cls_1[:, 0], cls_1[:, 1])
sns.scatterplot(cls_0[:, 0], cls_0[:, 1])


In [0]:
# for two circles dataset
# y_binary = to_categorical(labels)
# X = data

# train_samples = 900
# test_samples = 100
# hidden_layers = 16

# for sanity check
y_binary = to_categorical(Y)
X = x

train_samples = 120
test_samples = 20
hidden_layers = 4

**Neural Network** <br>
with <br>
an input layer that has 2 neurons (activation function is *tanh*), <br>
one hidden layer with *hidden_layer* neurons (activation function is *tanh*) <br>
and an output layer with 1 neuron (activation function is *sigmoid*)<br>
<br>
optimizer: *SGD* and the loss is *binary crossentropy*

In [0]:
# define model
model = keras.models.Sequential()
model.add(keras.layers.Dense(units=2, input_dim=X.shape[1], activation='tanh'))
model.add(keras.layers.Dense(units=hidden_layers, input_dim=2, activation='tanh'))
model.add(keras.layers.Dense(units=1, activation='sigmoid'))

# define optimizer and compile model
sgd_optimizer = keras.optimizers.SGD(lr=0.001, decay=1e-7, momentum=0.9)
model.compile(optimizer=sgd_optimizer, loss='binary_crossentropy')

# fitting the net
history = model.fit(X[:train_samples, :], y_binary[:train_samples, 0], batch_size=64, epochs=1000, verbose=2, validation_split=0.1)

# saving the weights for compering them with the bayesian_nn ones 
nn_summary = model.summary()
nn_weights = model.get_weights()


In [0]:
# evaluate model
score = model.evaluate(X[train_samples:, :], y_binary[train_samples:, 0])
print('acc: ', 1-score)

**Setting prior on weight for the bayesian neural network**

In [0]:

def prior_on_weights(X):
  # layers weights
  weight_lay1 = tf.convert_to_tensor(ed.Normal(name='w1', loc=np.zeros([X.shape[1], hidden_layers], dtype='float32'), 
                                               scale=np.ones([X.shape[1], hidden_layers],  dtype='float32')), dtype=tf.float32)
  weight_lay2 = tf.convert_to_tensor(ed.Normal(name='w2', loc=np.zeros([hidden_layers, 1],  dtype='float32'), 
                                               scale=np.ones([hidden_layers, 1],  dtype='float32')), dtype=tf.float32)
  # layers biases 
  bias_lay1 = tf.convert_to_tensor(ed.Normal(name='b1', loc=np.zeros([1, hidden_layers],  dtype='float32'), 
                                             scale=np.ones([1, hidden_layers],  dtype='float32')), dtype=tf.float32)
  bias_lay2 = tf.convert_to_tensor(ed.Normal(name='b2', loc=np.zeros(1,  dtype='float32'), 
                                             scale=np.ones(1,  dtype='float32')), dtype=tf.float32)
  
  # layers logic
  out_lay1 = tf.nn.tanh(tf.add(np.tensordot(X, weight_lay1, axes=1), bias_lay1))
  out_lay2 = keras.activations.sigmoid(np.tensordot(out_lay1, weight_lay2, axes=1)+ bias_lay2)

  # output layer as a Bernoulli disribution of second layer output
  output = ed.Bernoulli(name='out', logits=out_lay2[:, 0], dtype=tf.float32)
  return output


In [0]:
# initial values for weights
q_w1 = tf.random.normal([], mean=np.zeros([2, hidden_layers]), stddev=np.ones([2, hidden_layers]), dtype=tf.float32)
q_w2 = tf.random.normal([], mean=np.zeros([hidden_layers, 1]), stddev=np.ones([hidden_layers, 1]), dtype=tf.float32)

# initial values for biases
q_b1 = tf.random.normal([], mean=np.zeros([1, hidden_layers]), stddev=np.ones([1, hidden_layers]), dtype=tf.float32)
q_b2 = tf.random.normal([], mean=np.zeros(1), stddev=np.ones(1), dtype=tf.float32)

# convert train data to tensors
x_train = tf.convert_to_tensor(X[:train_samples, :], dtype=tf.float32)
y_train = tf.convert_to_tensor(y_binary[:train_samples, 0], dtype=tf.float32)


In [0]:
# define target probability distribution
log_joint = ed.make_log_joint_fn(prior_on_weights)

def target_log_prob_fn(w1, b1, w2, b2):
        return log_joint(x_train, w1=w1, b1=b1, w2=w2, b2=b2, out=y_train)


In [0]:
#  define method of sampling
 hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(target_log_prob_fn=target_log_prob_fn, step_size=0.01, num_leapfrog_steps=5)

In [0]:
# sample mcmc starting from initial values 
states = tfp.mcmc.sample_chain(num_results=  1000, current_state=[q_w1, q_b1, q_w2, q_b2], kernel=hmc_kernel, num_burnin_steps=500)


**Compute weights and biases as a mean over samples**

In [0]:
w1 = np.mean(states[0][kernel_results.is_accepted.numpy()], 0)
b1 = np.mean(states[1][kernel_results.is_accepted.numpy()], 0)
w2 = np.mean(states[2][kernel_results.is_accepted.numpy()], 0)
b2 = np.mean(states[3][kernel_results.is_accepted.numpy()], 0)

# print(w1)
# print(b1)
# print(w2)
# print(b2)


**Predict labels for test data**

In [0]:
x_test = tf.convert_to_tensor(X[train_samples:, :], dtype=tf.float32)
y1 = tf.math.tanh(tf.add(tf.matmul(x_test, w1), b1))
y2 = tf.math.sigmoid(tf.add(tf.matmul(y1, w2), b2))

y_predicted = y2.numpy()
y_predicted_classes = np.array([0 if y < .5 else 1 for y in y_predicted])


**Bayesian Neural Network Accuracy**



In [0]:
# !pip install sklearn
import sklearn.metrics as met

# define true labels for sanity check
true_labels = np.array([0 if y == False else True for y in y_binary[train_samples:, 0]])

# define true labels for two circles
# true_labels = labels[train_samples:]

print('bayesian_nn acc: ', met.accuracy_score(true_labels, y_predicted_classes))


**Compare the two approches**

In [0]:
print('nn_weights: ', nn_weights)
print('bnn_weights: ')
print('w1: ', w1)
print('b1: ', b1)
print('w2: ', w2)
print('b2: ', b2)

**Multiclass Classification Neural Network**

In [0]:
from sklearn.datasets import make_circles

t = make_circles(n_samples=1000, noise=0.04)
t2 = make_circles(n_samples=1000, noise=0.1, random_state=0, factor=0.5)

labels = t[1]
data = t[0]

labels2 = t2[1]
data2 = t2[0]

cls_1 = data[labels == 1]
cls_0 = data[np.logical_not(labels)]

cls_12 = data2[labels2 == 1]
cls_02 = data2[np.logical_not(labels2)]

sns.scatterplot(cls_1[:, 0], cls_1[:, 1])
sns.scatterplot(cls_0[:, 0], cls_0[:, 1])
sns.scatterplot(cls_12[:, 0], cls_12[:, 1])


In [0]:
dataset = pd.DataFrame(data)
dataset['labels'] = labels
aux = pd.DataFrame(data2[labels2 == 1])
aux['labels'] = 2
dataset = dataset.append(aux)


In [0]:

n_samples = dataset.shape[0]
n_features = dataset.shape[1]
n_classes = len(set(dataset.labels))
n_hidden = 16

In [0]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing as pr
data_train, data_test, labels_train, labels_test = train_test_split(pr.scale(np.array(dataset.iloc[:, :2])), np.array(dataset.labels), test_size=0.2)


In [0]:
# define model
model2 = keras.models.Sequential()

model2.add(keras.layers.Dense(units=n_features, activation='relu'))
model2.add(keras.layers.Dense(units=n_hidden, activation='relu'))
model2.add(keras.layers.Dense(units=n_classes, activation='softmax'))
 
# define optimizer and compile model
sgd_optimizer = keras.optimizers.SGD(lr=0.001,decay=1e-6, momentum=0.9)
model2.compile(optimizer=sgd_optimizer, loss='sparse_categorical_crossentropy')

# fitting the net
history = model2.fit(data_train, labels_train, epochs=1200, verbose=2, validation_split=0.1, batch_size=64)

# saving the weights for compering them with the bayesian_nn ones 
nn2_summary = model2.summary()
nn2_weights = model2.get_weights()

In [0]:
# print(labels_test.shape)
score2 = model2.evaluate(data_test, labels_test)
print('acc: ', 1-score2)

In [0]:

def prior_on_weights_multiclass(X):
  
  # layers weights
  weight_lay1 = tf.convert_to_tensor(ed.Normal(name='w1', loc=np.zeros([2, n_hidden], dtype='float32'), 
                                               scale=np.ones([2, n_hidden],  dtype='float32')), dtype=tf.float32)
  weight_lay2 = tf.convert_to_tensor(ed.Normal(name='w2', loc=np.zeros([n_hidden, n_classes],  dtype='float32'), 
                                               scale=np.ones([n_hidden, n_classes],  dtype='float32')), dtype=tf.float32)
  # layers biases 
  bias_lay1 = tf.convert_to_tensor(ed.Normal(name='b1', loc=np.zeros([1, n_hidden],  dtype='float32'), 
                                             scale=np.ones([1, n_hidden],  dtype='float32')), dtype=tf.float32)
  bias_lay2 = tf.convert_to_tensor(ed.Normal(name='b2', loc=np.zeros(n_classes,  dtype='float32'), 
                                             scale=np.ones(n_classes,  dtype='float32')), dtype=tf.float32)
  
  # layers logic
  out_lay1 = tf.nn.relu(tf.add(np.tensordot(X, weight_lay1, axes=1), bias_lay1))
  out_lay2 = keras.activations.softmax(np.tensordot(out_lay1, weight_lay2, axes=1)+ bias_lay2)

  # output layer as a Categorical disribution of second layer output
  output = ed.Categorical(name='out', logits=out_lay2, dtype=tf.float32)
  return output


In [0]:
# initial values for weights
q_w1 = tf.random.normal([], mean=np.zeros([2, n_hidden]), stddev=np.ones([2, n_hidden]), dtype=tf.float32)
q_w2 = tf.random.normal([], mean=np.zeros([n_hidden, n_classes]), stddev=np.ones([n_hidden, 1]), dtype=tf.float32)

# initial values for biases
q_b1 = tf.random.normal([], mean=np.zeros([1, n_hidden]), stddev=np.ones([1, n_hidden]), dtype=tf.float32)
q_b2 = tf.random.normal([], mean=np.zeros(n_classes), stddev=np.ones(n_classes), dtype=tf.float32)

# convert train data to tensors
x_train = tf.convert_to_tensor(data_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(labels_train, dtype=tf.float32)
print(y_train)

In [0]:
# define target probability distribution
log_joint = ed.make_log_joint_fn(prior_on_weights_multiclass)

def target_log_prob_fn(w1, b1, w2, b2):
        return log_joint(x_train, w1=w1, b1=b1, w2=w2, b2=b2, out=y_train)


In [0]:
#  define method of sampling
 hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(target_log_prob_fn=target_log_prob_fn, step_size=0.01, num_leapfrog_steps=5)

In [0]:
# sample mcmc starting from initial values 
states = tfp.mcmc.sample_chain(num_results=100, current_state=[q_w1, q_b1, q_w2, q_b2], kernel=hmc_kernel, num_burnin_steps=50)


In [0]:
w1 = np.mean(states[0][kernel_results.is_accepted.numpy()], 0)
b1 = np.mean(states[1][kernel_results.is_accepted.numpy()], 0)
w2 = np.mean(states[2][kernel_results.is_accepted.numpy()], 0)
b2 = np.mean(states[3][kernel_results.is_accepted.numpy()], 0)

print(w1)
print(b1)
print(w2)
print(b2)

# print(len(states))

In [0]:

l1 = tf.nn.relu(tf.add(tf.matmul(tf.convert_to_tensor(data_test, dtype=tf.float32), w1), b1))
l2 = tf.math.softmax(tf.add(tf.matmul(l1, w2), b2))

y_predicted = y2.numpy().argmax(axis=1)


In [0]:
print(y_predicted)

In [0]:
# !pip install sklearn
import sklearn.metrics as met

print('bayesian_nn_multiclass acc: ', met.accuracy_score(labels_test, y_predicted))
